# Lab 7 |&nbsp;&nbsp;The knights of the "square" table

 &copy;&nbsp; murina 2020 

Today is the annual knigth meeting in the Real Palace. There are a square table and s seats for p<s persons. All the people must to keep distance in order to stop the spread of the Covid-19.<br>
Describe the situation as a computational problem and encode it in a QUBO representation. To make things easier, only think of the case of an even number of knights.

### Solution

<br>
<br>

#### Encoding the scenario in bits


bit => seat<br>
|0| => seat: available<br>
|1| => seat: occupied<br>
<br>
So, we are using a QUBO representation for the computational problem.<br>
<br>
Example with p=6 and s=8: 
<br>
<br>
<br>

## <ins>>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;0&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;0&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>

<br>
<br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; |0|&emsp;&emsp;&emsp;|1|<br>
&emsp;&emsp;&emsp;&emsp;x &nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x<br>
&emsp;&emsp;&emsp;&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x<br>
&emsp;&emsp;|1|&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x&emsp;|1|<br>
&emsp;&emsp;&emsp;&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x<br>
&emsp;&emsp;|1|&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x&emsp;|1|<br>
&emsp;&emsp;&emsp;&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x<br>
&emsp;&emsp;&emsp;&emsp;x &nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; >|1|&emsp;&emsp;&emsp;|0|<br>
<br>
<br>

#### Encoding the computational problem as an interaction model in QUBO representation

For the meeting we need all the knights sitting:
<br>
<br>
1) all the people sitting:
<br>
$$(\sum_{i} x_{i}-p)^{2} = 0$$

<br>
<br>
<br>
$$=(1-2p)\sum_{i} x_{i}+2\sum_{<i,j>}x_{i}x_{j}+p^{2}$$
<br>
<br>
In order to keep distance, we should avoid:<br>
<br>
<br>
2) people sitting on both sides of each knight<br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\textrm{addition of the penalty}  \hspace{2cm} \sum_{i}      m{\hspace{0.1cm}}x_{i}x_{i+1} \hspace{2cm} m\in{\mathbb {R}>0} $
<br>
<br>
<br>
<br>
3) people sitting in front of each knight <br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\textrm{addition of the penalty} \hspace{2cm}\sum_{i}  l{\hspace{0.1cm}}x_{i}x_{i+j(i)} \hspace{2cm} l\in{\mathbb {R}>0} \hspace{2cm}$<br>
<br>

$$j(i) \hspace{0.3cm}\textrm{is the index of the seat in front of seat i}$$

<br>
<br>


#### Tuning the Lagrance multipliers $\lambda_{i}$

<br>

## $$\lambda_{0}(\sum_{i} x_{i}-2p)^{2} + \lambda_{1}\sum_{i}m{\hspace{0.1cm}}x_{i}x_{i+1}+ \lambda_{2}\sum_{i}l{\hspace{0.1cm}}x_{i}x_{i+j(i)}= 0$$



### 8 seats for 4 knights 

####  QUBO scalar representation (coding h and J)
<br>

In [164]:
import dimod  

from collections import defaultdict 

seats = 4 * 4
persons = seats/2 - 1

h = defaultdict(int)

J = defaultdict(int)

m = 1

l = 1


lambda_0 = 1.5


e_offset = lambda_0 * persons**2

for _ in range(seats):
    
    h[ _ ] = lambda_0 * ( 1 - 2*persons )   

for _ in range(seats - 1):
    for __ in range( _ + 1, seats):
        J[ _ , __ ] =  lambda_0*2    
    

lambda_1 = 1   
    
for _ in range(seats-1):
    J[ _ , _ + 1] += lambda_1 * m
    
J[0, seats - 1] += lambda_1 * m


lambda_2 = 3

quarter_period = int(seats/4)

for _ in range(quarter_period):
    J[ _ , 3*quarter_period -1 - _ ] = lambda_2 * l
    
for _ in range(quarter_period, 2*quarter_period):
    J[ _ , 5*quarter_period-1 - _ ] = lambda_2 * l




bqm = dimod.BinaryQuadraticModel(h,J,e_offset,'BINARY')   #dimod.BINARY)

results = dimod.ExactSolver().sample(bqm)



In [165]:
smpl = results.first.sample

energy = results.first.energy

quarter_period = int(seats/4)


spam = "            "

egg = "          "

for _ in reversed(range(3*quarter_period, 4*quarter_period)):
    spam += str(smpl[_]) + "     "
    egg += "* * * "
print(spam) 
print(egg)

print("       *", quarter_period*"      ", "*  ")

egg = "          "

for _ in range(quarter_period):
    print(smpl[_],"     *", quarter_period*"      ", "*  ",smpl[_ + quarter_period -1])   
    print("       *", quarter_period*"      ", "*  ")  
    egg += "* * * "

print("       *", quarter_period*"      ", "*  ")
print(egg)

spam = "            "

for _ in range(quarter_period, 2*quarter_period):
    spam += str(smpl[_]) + "     "
print(spam) 

            0     1     0     0     
          * * * * * * * * * * * * 
       *                          *  
1      *                          *   0
       *                          *  
0      *                          *   1
       *                          *  
1      *                          *   0
       *                          *  
0      *                          *   1
       *                          *  
       *                          *  
          * * * * * * * * * * * * 
            1     0     1     0     


In [166]:
e_lambda0 = 0.

for _ in range(len(smpl)):
    e_lambda0 += smpl[_]

e_lambda0 += -2*persons 

energy_e_lambda0 = lambda_0 * ( e_lambda0**2 - persons**2 )

print(energy_e_lambda0)

e_lambda1 = smpl[0]*smpl[len(smpl)-1]


for _ in range(len(smpl)-1):
    e_lambda1 += smpl[_]*smpl[_+1]

energy_e_lambda1 = lambda_1*m*e_lambda1 

print(energy_e_lambda1)


e_lambda2 = 0

for _ in range(quarter_period):
    e_lambda2 += smpl[ _ ] * smpl[ 3*quarter_period -1 - _ ]  
    
for _ in range(quarter_period, 2*quarter_period):
    e_lambda2 += smpl[ _ ] * smpl[ 5*quarter_period-1 - _ ]
    
energy_e_lambda2 = lambda_2*l*e_lambda2 

print(energy_e_lambda2)



0.0
0
0


 ### 12 seats for 6 knights

#### QUBO matrix representation

In [1]:
import dimod

from collections import defaultdict


seats = 12
persons = 6

J = defaultdict(int)


#####################################################################################################
# PAY ATTENTION: now, we DO apply all the conditions 1), 2), and 3) to optain a correct output      
##################################################################################################

lambda_0 = 1.2 


e_offset = lambda_0 * persons**2

for _ in range(seats):
    
    J[ _ , _ ] = lambda_0 * ( 1 - 2*persons )   

for _ in range(seats - 1):
    for __ in range( _ + 1, seats):
        J[ _ , __ ] =  lambda_0*2    
    

lambda_1 = 1   
    
for _ in range(seats-1):
    J[ _ , _ + 1] += lambda_1
    
J[0, seats - 1] += lambda_1 
    
    
lambda_2 = 1
    
J[0,8] += lambda_2 
J[1,7] += lambda_2    
J[2,6] += lambda_2    
J[3,11] += lambda_2
J[4,10] += lambda_2
J[5,9] += lambda_2




print(J)
# Correct approach

bqm = dimod.BinaryQuadraticModel.from_qubo(J, offset= e_offset)

results = dimod.ExactSolver().sample(bqm)

print(results)

defaultdict(<class 'int'>, {(0, 0): -13.2, (1, 1): -13.2, (2, 2): -13.2, (3, 3): -13.2, (4, 4): -13.2, (5, 5): -13.2, (6, 6): -13.2, (7, 7): -13.2, (8, 8): -13.2, (9, 9): -13.2, (10, 10): -13.2, (11, 11): -13.2, (0, 1): 3.4, (0, 2): 2.4, (0, 3): 2.4, (0, 4): 2.4, (0, 5): 2.4, (0, 6): 2.4, (0, 7): 2.4, (0, 8): 3.4, (0, 9): 2.4, (0, 10): 2.4, (0, 11): 3.4, (1, 2): 3.4, (1, 3): 2.4, (1, 4): 2.4, (1, 5): 2.4, (1, 6): 2.4, (1, 7): 3.4, (1, 8): 2.4, (1, 9): 2.4, (1, 10): 2.4, (1, 11): 2.4, (2, 3): 3.4, (2, 4): 2.4, (2, 5): 2.4, (2, 6): 3.4, (2, 7): 2.4, (2, 8): 2.4, (2, 9): 2.4, (2, 10): 2.4, (2, 11): 2.4, (3, 4): 3.4, (3, 5): 2.4, (3, 6): 2.4, (3, 7): 2.4, (3, 8): 2.4, (3, 9): 2.4, (3, 10): 2.4, (3, 11): 3.4, (4, 5): 3.4, (4, 6): 2.4, (4, 7): 2.4, (4, 8): 2.4, (4, 9): 2.4, (4, 10): 3.4, (4, 11): 2.4, (5, 6): 3.4, (5, 7): 2.4, (5, 8): 2.4, (5, 9): 3.4, (5, 10): 2.4, (5, 11): 2.4, (6, 7): 3.4, (6, 8): 2.4, (6, 9): 2.4, (6, 10): 2.4, (6, 11): 2.4, (7, 8): 3.4, (7, 9): 2.4, (7, 10): 2.4, (7, 11

In [6]:
smpl = results.first.sample

energy = results.first.energy

print(smpl)
print("         ",smpl[11] , "   ", smpl[10], "   ", smpl[9])
print("         * * * * * * * * *   ")

print("       *                   *")
print(smpl[0],"     *                   *  ",smpl[8])
print("       *                   *")
print(smpl[1],"     *                   *  ",smpl[7])
print("       *                   *")
print(smpl[2],"     *                   *  ",smpl[6])
print("       *                   *")
print("         * * * * * * * * *   ")
print("         ",smpl[3],"   ",smpl[4], "   ", smpl[5])

{0: 1, 1: 0, 2: 1, 3: 0, 4: 1, 5: 0, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1}
          1     0     1
         * * * * * * * * *   
       *                   *
1      *                   *   0
       *                   *
0      *                   *   1
       *                   *
1      *                   *   0
       *                   *
         * * * * * * * * *   
          0     1     0
